背景：

    神经网络的学习，通过数值微分计算了神经网络的权重参数的梯度（严格来说，是损失函数关于权重参数的梯度）。

    数值微分虽然简单，也容易实现，但是计算速度比较慢。接下来学习一个高效计算权重参数的方法----误差反向传播法。

    理解误差反向传播法的两种方法：
     
     1、数学式
     2、 computational graph 计算图

计算图流程
     
     1、构建计算图
     2、在计算图上做正向传播：从计算图出发点到结束点的传播
     3、在计算图上做反向传播：从计算图结束点到出发点的传播

局部计算

    计算图的特征是可以通过传递“局部计算”获得最终结果。

        局部：与自己相关的某个小范围
        局部计算：无论全局发生了什么，都能只根据与自己相关的信息输出接下来的结果。

计算图优点：

计算图可以集中精力于局部计算，无论全局的计算有多么复杂，各个步骤所要做的就是对象节点的局部计算。虽然局部计算非常简单，但是通过传递它的计算结果，可以获得全局的复杂计算的结果。

利用计算图可以将中间的计算结果全部保存起来

以通过正向传播和反向传播高效地计算各个变量的导数值

反向传播中传递局部导数的原理，是基于链式法则的：

